In [1]:
import biom
from gneiss.balances import sparse_balance_basis
import numpy as np
import pandas as pd
from bp import parse_newick, to_skbio_treenode

In [2]:
micro_table = biom.load_table('../data/hcc-data/46119_otu_table.biom').to_dataframe().T
metab_table = biom.load_table('../data/hcc-data/metabolites.biom').to_dataframe().T
metadata = pd.read_table('../data/sample_information_from_prep_2458.tsv', index_col=0)

metabolite_md = pd.read_table('../data/hcc-data/metabolite_feature_metadata.txt', index_col=0)

In [3]:
metadata = metadata.reset_index().set_index('host_subject_id')
metadata = metadata.loc[metab_table.index]
micro_table = micro_table.loc[metadata['sample_id'].values]
micro_table.index = metadata.index
metab_table = metab_table.loc[metadata.index]

# drop infrequent taxa
micro_table = micro_table.loc[:, (micro_table > 0).sum(axis=0) >= 10]

In [4]:
# phylogenetic tree for microbes
gg_dir='/Users/mortonjt/Documents/research/databases/gg/gg_13_5_otus/trees'
bp_tree = parse_newick(open(f'{gg_dir}/97_otus.tree').read())
bp_tree2 = bp_tree.shear(set(micro_table.columns))
sktree = to_skbio_treenode(bp_tree2)
sktree.prune()

# rename internal nodes
i = 0
for n in sktree.levelorder():
    if n.name is None:
        n.name = f'clade{i}'
        i += 1

In [5]:
common_ids = list(set(metab_table.columns) & set(metabolite_md.index))
metab_table = metab_table[common_ids]
metabolite_md = metabolite_md.loc[common_ids]

In [6]:
metabolite_md['mz'] = list(map(lambda x: float(x.split('_')[0]), metabolite_md.index))

In [7]:
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage
from skbio import TreeNode
from gneiss.util import rename_internal_nodes, match_tips

dm = pdist(metabolite_md['mz'].values.reshape(-1, 1))
lm = linkage(dm, 'average')
met_tree = TreeNode.from_linkage_matrix(lm, metabolite_md.index)
met_tree = rename_internal_nodes(met_tree)

In [8]:
micro_table, sktree = match_tips(micro_table, sktree)
metab_table, met_tree = match_tips(metab_table, met_tree)

Combine trees

In [48]:
from skbio.stats.composition import clr, multiplicative_replacement

In [53]:
micro_clr = pd.DataFrame(clr(micro_table + 1),
                         index=micro_table.index,
                         columns=micro_table.columns)

metab_clr = pd.DataFrame(clr(metab_table + 1),
                         index=metab_table.index,
                         columns=metab_table.columns)

In [55]:
combined_tree = TreeNode()
combined_tree.append(sktree)
combined_tree.append(met_tree)
combined_tree.name = 'Head'
combined_table = pd.concat((micro_clr, metab_clr), axis=1)

In [56]:
from gneiss.balances import sparse_balance_basis
Psi, nodes = sparse_balance_basis(combined_tree)

In [57]:
combined_table.shape, Psi.shape

((438, 13364), (13363, 13364))

In [58]:
combined_ilr = pd.DataFrame(combined_table.values @ Psi.T,
                            index=combined_table.index,
                            columns=nodes)

In [64]:
micro_nodes = [n.name for n in sktree.levelorder(include_self=True) if not n.is_tip()]
metab_nodes = [n.name for n in met_tree.levelorder(include_self=True) if not n.is_tip()]

In [70]:
microV = combined_ilr[micro_nodes].var(axis=0).sum()
metabV = combined_ilr[metab_nodes].var(axis=0).sum()

In [71]:
combined_ilr.to_csv('../data/combined/balances.csv')
combined_tree.write('../data/combined/tree.nwk')
metadata.to_csv('../data/combined/metadata.csv')

In [74]:
rescaled_ilr = pd.concat((combined_ilr[micro_nodes] / microV, 
                          combined_ilr[metab_nodes] / metabV), axis=1)

In [75]:
rescaled_ilr.to_csv('../data/combined/rescaled_balances.csv')

In [79]:
combined_ilr

,Head,clade0,y0,clade1,clade2,y1,y2,p__Cyanobacteria,clade3,y3,...,clade1388,clade1389,clade1390,clade1391,clade1392,clade1393,clade1394,clade1395,clade1396,clade1397
SS11,-1.381137e-11,-0.717217,-0.025557,-0.203422,1.612685,-13.355241,-4.143765,9.540934e-01,2.868638,0.540313,...,-0.897013,0.000000,2.520608,-0.260021,0.000000,-1.064888e+00,-1.553672,-1.138044,-3.011957,0.000000
SS12,-1.047870e-12,-0.308204,4.841941,1.266965,0.596101,-12.825570,-5.583822,-5.088209e-01,3.118053,-3.464502,...,-1.131905,0.000000,1.676581,2.303822,0.000000,-1.105234e+00,-1.960516,-1.757094,-3.126073,0.000000
SS13,8.124388e-12,-1.206063,4.238371,0.490129,-1.085233,-15.461007,0.994771,-1.315281e+00,4.083631,0.129265,...,-1.202062,0.000000,2.927346,0.508497,0.000000,-1.282483e+00,-2.082033,-1.470387,-3.627410,0.000000
SS14,5.654512e-12,0.163962,-7.022320,-0.490129,1.392174,-3.326803,-3.877166,4.163336e-17,2.415509,6.249535,...,-1.179989,0.980258,2.749386,3.829761,0.000000,-2.000944e-01,-2.043801,0.000000,-0.565952,0.000000
SS15,-1.091257e-11,0.625053,4.465891,0.000000,1.331068,-10.793756,-1.968727,4.163336e-17,1.961105,5.111717,...,-1.678257,0.490129,1.462965,2.217129,0.000000,-5.551115e-17,-2.906827,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SS95,2.151043e-12,0.517003,-3.846732,0.000000,1.100971,-0.758739,5.463083,-4.163336e-17,2.172898,3.364793,...,0.000000,0.000000,0.897013,2.858869,0.000000,-9.051392e-01,0.000000,0.000000,-2.560120,0.000000
SS96,-3.989610e-12,0.003831,-1.312363,0.000000,0.008159,-1.823080,-3.309286,-3.252607e-19,-0.236406,1.126359,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
SS97,-7.830098e-12,0.034353,2.889023,0.000000,0.073155,-10.935165,-6.153127,4.336809e-18,-1.576338,4.738524,...,0.000000,0.000000,0.000000,0.000000,0.000000,-1.734723e-18,0.000000,0.000000,0.000000,0.000000
SS98,-1.453292e-11,0.531082,-8.255852,0.000000,1.130953,-5.044111,-5.567751,1.387779e-17,0.695335,5.087088,...,0.000000,0.000000,0.282976,3.107345,0.000000,-1.535209e+00,0.000000,-0.490129,-3.493298,0.490129
